## アッシェンフェルターのワイン方程式

ワインの格付け評価で有名なものに[パーカーポイント（ワインショップによる解説）](https://cocos.co.jp/blog/law/parker-point/)があります．
そこにデータサイエンスの方向から新たな視点を提供したのが，経済学者アッシェンフェルターでした．
彼は，データ分析にもとづきボルドーワインの出来を推定し，中にはパーカーポイントへの疑義となる「86年ものボルドーは水っぽいはずだ」のような予測も導きました．
実際，86年を高評価したパーカーの予想は外れます．
その方程式とは次のようなものです．
$$
ワインの品質 = 12.145 + 0.00117\times 冬の降雨量
+ 0.0614 \times 育成期平均気温
- 0.00386\times 収穫期降雨量
$$
式の変数を見ればわかるように，ワインそのものの情報ではなく，ブドウの出来を予測するものと言ってよいでしょう．
一定の技術のあるワイナリーで醸造するワインは，ブドウの出来で決まるというのです．参考文献：「その数学が戦略を決める（Super Crunchers）」，イアン・エアーズ著，　[山形浩生訳](https://cruel.hatenablog.com/entry/20150121/1421802947)

中でもパーカーにできなくて，方程式にできたことがあります．
すでにブドウは収穫されても，まだワインとして熟成を経ていない将来の格付け情報を予測することでした．
実際，発表当時に方程式が導いた89年および90年ものの予測は，方程式による予測への信頼を得るものとなりました．

方程式を導くデータ分析の例が以下のものです．
分析のためのデータは，[アッシェンフェルター自身によるもの](http://www.liquidasset.com/winedata.html)にあり，
これをダウンロードして利用します．
このデータを用いた[アッシェンフェルター自身による分析論文](http://www.liquidasset.com/orley.htm)も読めます．

最初にアッシェンバッハのデータ（CSVファイルで用意）を読み込み，データの確認（Head関数）および各列間の散布図および相関係数を見ます．
各列はワインの価格，ある時期の降水量，気温であり，上の方程式と見比べればなんとなく想像はつくでしょう．

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ashenfelterのデータ読み込み
WineData = pd.read_csv("ashenfelter.csv")
# cofirm each columns
WineData.head()

# 散布図
pd.plotting.scatter_matrix(WineData)

次に重回帰分析を行います．sci-kit learnを使います．以下のプログラムではエラーを生じます．Head関数でデータを表示させたときに，いくつかの項目は数値データに欠けていました．
これを欠損値といいます．




In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, LeaveOneOut

WineData.head()

# 価格以外をXに
X, y = WineData.drop(columns=['LPRICE2']), WineData['LPRICE2']
WineModel = LinearRegression().fit(X, y)

次のプログラムではDrop関数によりエラーの原因となった欠損値を取り除いています．これにより実行処理できるようになります．
Head関数で1956年データ行のなくなったことを確認してみてください．
最後に係数を表示させます．
最初にあげたアッシェンフェルターの方程式と合致したでしょうか．
確認してみましょう．

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, LeaveOneOut

WineData2 = WineData.dropna()
WineData2.head()

# 価格以外をXに
X, y = WineData2.drop(columns=['LPRICE2']), WineData2['LPRICE2']
WineModel = LinearRegression().fit(X, y)

# 係数表示，切片を先に実行
WineModel.intercept_
pd.Series(WineModel.coef_, index=X.columns)

## さいごに

各列は何かわかるでしょう，といいました．
VINT列はヴィンテージを表します．
ヴィンテージで価格が決まるのなら，この変数の影響が一番大きくなるのではないでしょうか．
ところが，上で求めた係数は，最初にあげた方程式にほぼ同じです．

これは，アッシェンフェルターの方程式はヴィンテージ情報がなくとも価格を予想できるという証左になっているということです．

以下のコードは，この考えの確認に，変数VINTも除いて実行しました．

In [ ]:

# 価格以外をXに
X, y = WineData2.drop(columns=['LPRICE2', 'VINT']), WineData2['LPRICE2']
WineModel = LinearRegression().fit(X, y)

# 係数表示，切片を先に実行
WineModel.intercept_
pd.Series(WineModel.coef_, index=X.columns)